# Deliverable 1
### Research Question
Based on performance metrics, are some football players overvalued?

Hypothesis:
- Players with higher goal-scoring rates have higher valuations, and those who are valuations far exceeding the indication given by the correlation are overvalued.

Performance metrics available:
- Goals scored per game
- Assists per game
- Minutes played
- Matches starting eleven
- Number of matches team captain
- Clean sheets (for goalkeepers) (!!We can leave goalkeepers out, since their value evaluation metrics do not correspond to those of other player positions)
- Age 
- Position (except goalkeeper since those have no goal and little to no assists)
- Foot
- Height

Required datasets:
- players.csv (to combine player-ids to actual names, birthcountry(?), age, position, foot, height, market value, highest market value)
- game_lineups.csv (to get whether the player was starting and if he was team captain (maybe redundant))
- appearances.csv (for each player appearance we get goals, assists, minutes_played)
- clubs.csv (to combine club-id to actual club names) (needed?)

# Deliverable 2

Import data

In [1]:
# Import packages
import pandas as pd, os, sys

# Load datasets
cwd = sys.path[0]
players = pd.read_csv(cwd+'/archive/players.csv')
lineups = pd.read_csv(cwd+'/archive/game_lineups.csv')
appearances = pd.read_csv(cwd+'/archive/appearances.csv')
clubs = pd.read_csv(cwd+'/archive/clubs.csv')

We can combine "lineups" with "appearances", since those are match-based, and we will add the number-of-appearances-column to the "players" dataset.

First we clean and preprocess the separate datasets by doing the following:

- Look for contradictions in the datasets "lineups" and "appearances"

- We join datasets on `game_id` and `player_id`

In [ ]:
# First, we inspect the dataframe

print(appearances.isna().sum())
print(lineups.isna().sum())

print(lineups.shape, appearances.shape) #(2191911, 10) (1578761, 13)
game_stats_na = lineups.merge(appearances, how='inner', on=['player_id', 'game_id'])
print(game_stats_na.info())

game_stats_na['player_name_y'] = game_stats_na['player_name_y'].fillna(game_stats_na['player_name_x'])
game_stats = game_stats_na
print(game_stats.info())

# Since `date` and `player_name` should also be the same for both datasets we can look for contradictions now we have resolved all null values

contradicting_dates_count = (game_stats['date_x'] != game_stats['date_y']).sum()

print(f"Number of rows with non-matching dates: {contradicting_dates_count}")

contradicting_names_count = (game_stats['player_name_x'] != game_stats['player_name_y']).sum()

print(f"Number of rows with non-matching names: {contradicting_names_count}")


# The data contains no contradictions for the dates, however it does contain 31986 contradictions for the names

contradicting_names = game_stats[game_stats['player_name_x'] != game_stats['player_name_y']]
print(f"Unique contradicting names: {contradicting_names[['player_name_x', 'player_name_y']].nunique()}")

# We have 640 unique names that are contradicting

Solve contradictions by character standardization

In [ ]:
# convert to string
players['first_name'] = players['first_name'].astype(str)
players['last_name'] = players['last_name'].astype(str)
players['name'] = players['name'].astype(str)

In [ ]:
char_replacement = {
    '-': ' ',
    'ö': 'o',
    'ó': 'o',
    'ò': 'o',
    'í': 'i',
    'é': 'e',
    'ä': 'a',
    'ü': 'u',
    'ß': 'ss',
    'å': 'a',
    'ø': 'o',
    'ñ': 'n',
    'ç': 'c',
    'œ': 'oe',
    'æ': 'ae',
    'ė': 'e',
    'ż': 'z',
    'ł': 'l',
    'č': 'c',
    'ś': 's',
    'ź': 'z',
    'ñ': 'n',
    'ã': 'a',
    'į': 'i',
    'š': 's',
    'ž': 'z',
    'đ': 'd',
    'ć': 'c',
    'ț': 't',
    'ğ': 'g',
    'ş': 's',
    'î': 'i',
    'ă': 'a',
    'Ș': 'S',
    'Ț': 'T',
    'İ': 'I',
    'ı': 'i',
    'ё': 'e',
    'й': 'i',
    'ю': 'u',
    'я': 'ya',
    'ë': 'e',
    'ș': 's',
    'ţ': 't',
    'ï': 'i',

}

def replace_special_chars(text, replacements):
    for special_char, normal_char in replacements.items():
        text = text.lower().replace(special_char, normal_char)
    return text

game_stats['player_name_x'] = game_stats['player_name_x'].apply(replace_special_chars, args=(char_replacement,))
game_stats['player_name_y'] = game_stats['player_name_y'].apply(replace_special_chars, args=(char_replacement,))

# Replace special characters in the first and last name columns of the players dataframe
players['first_name'] = players['first_name'].apply(replace_special_chars, args=(char_replacement,))
players['last_name'] = players['last_name'].apply(replace_special_chars, args=(char_replacement,))
players['name'] = players['name'].apply(replace_special_chars, args=(char_replacement,))

contradicting_names_count = (game_stats['player_name_x'] != game_stats['player_name_y']).sum()
contradicting_names = game_stats[game_stats['player_name_x'] != game_stats['player_name_y']]
print(f"Number of rows with non-matching names: {contradicting_names_count}")
print(contradicting_names[['player_name_x', 'player_name_y']].nunique())

After cleaning capitals and special characters we still have 166 contradictions left. After inspection we can conclude these contradictions are regarding the inclusion of middle names, no first name or no second name.

Since these do not directly affect our results we have chosen to make player_name_x the leading column since it is the most inclusive.

In [ ]:
game_stats_cleaned = game_stats.drop(['player_name_y', 'date_y'], axis=1).rename(columns={'player_name_x':'player_name', 'date_x':'date'})

Looking for conflicts such as duplicate entries

In [ ]:
duplicates = appearances.duplicated(subset=['player_id', 'game_id'], keep=False)
duplicate_entries = appearances[duplicates]

# We have 0 duplicate entries

appearance_counts = game_stats_cleaned.groupby('player_id')['player_id'].count().reset_index(name='number_of_appearances')

result_players = pd.merge(players, appearance_counts, on='player_id', how='left')

Since we are interested in the market value, we can remove all players for which we do not have an appearance count, because those players are not included in the metrics dataset.

In [ ]:
result_players_clean = result_players.dropna(axis=0, how='any')

In [ ]:
# Convert number of appearances to integer
result_players_clean['number_of_appearances'] = result_players_clean['number_of_appearances'].astype(int)

In [ ]:
# Drop unnecessary columns
result_players_clean = result_players_clean.drop(['player_code', 'city_of_birth', 'country_of_citizenship', 'agent_name', 'image_url', 'url'], axis=1)
game_stats_cleaned = game_stats_cleaned.drop(['number'], axis=1)

# Change name of column name to player_name
result_players_clean = result_players_clean.rename(columns={'name':'player_name', 'current_club_id':'player_current_club_id'})

In [ ]:
game_stats_cleaned = game_stats_cleaned.merge(result_players_clean, on='player_id', how='inner')

Saving the preprocessed data

In [ ]:
os.makedirs(cwd+'/clean', exist_ok=True)
game_stats_cleaned.to_csv(cwd+'/clean/game_stats_cleaned.csv', index=False)
result_players_clean.to_csv(cwd+'/clean/result_players_clean.csv', index=False)

# Deliverable 3
As part of the poster, we need to attempt to deliver the answer to the research question. Therefore a knowledge graph is created here.

In [ ]:
# Load datasets
import sys, pandas as pd
cwd = sys.path[0]
game_stats_cleaned = pd.read_csv(cwd+"/clean/game_stats_cleaned.csv") # index_col=0
result_players_clean = pd.read_csv(cwd+"/clean/result_players_clean.csv") # index_col=0

/tmp/ipykernel_24115/3664397204.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  game_stats_cleaned = pd.read_csv(cwd+"/clean/game_stats_cleaned.csv", index_col=0)


In [ ]:
from rdflib import Graph, URIRef, Literal, BNode, Namespace, RDF

n = Namespace(URIRef("http://example.org/"))
kg = Graph()
kg.bind("ex", n)

# Append player_id to knowledge graphs
for player_id in game_stats_cleaned.player_id.unique():
    # Create subset df
    player_df = game_stats_cleaned[game_stats_cleaned.player_id==player_id]
    # Append player_id to knowledge graph
    player_node = URIRef(n + str(player_id))
    kg.add((player_node, RDF.type, n.player))
    # Append player name to player_id
    player_name = player_df.player_name.iloc[0]
    kg.add((player_node, n.player_name, Literal(player_name)))
    # Append games to player_id
    for game_id in player_df.game_id:
        game_node = BNode()
        kg.add((game_node, RDF.type, n.game))
        kg.add((player_node, n.played_in_game, game_node))
        kg.add((game_node, n.id, Literal(game_id)))
        # Append attributes to game_id
        for attr, value in player_df[player_df.game_id==game_id].squeeze().items():
            kg.add((game_node, n[attr], Literal(value)))
    # Append attributes from result_players_clean to player_id
    for attr, value in result_players_clean[result_players_clean.player_id==player_id].squeeze().items():
        kg.add((player_node, n[attr], Literal(value if not isinstance(value,pd.Series) else None)))
# Save
kg.serialize(cwd+"/knowledge_graph.ttl", format="turtle")

<Graph identifier=N3cc847a20f294fda8e432376dfc91376 (<class 'rdflib.graph.Graph'>)>